In [1]:
if "zmqshell" in str(get_ipython()):
    print("Running on MIRP")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido"
else:
    import pathlib
    temp = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
    print("Running local")

Running on MIRP


In [2]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import datetime
import gc
import pandas as pd
import joypy

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler as Normalizer

import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.utils.data import random_split

import torchvision.datasets as datasets

from Zamuro_DataLoader import SoundscapeData
from Models import ConvAE as AE
from AE_training_functions import TestModel, TrainModel
from AE_Clustering import AE_Clustering 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from datetime import timedelta

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap

import random
def _set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).
 
    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
_set_seed(1024)

/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/umap/dist

In [3]:
# PCA_ = PCA(n_components=60).fit(X_norm)
# X_PCA = PCA_.transform(X_norm)
# # X_TSNE = TSNE(n_components=60, learning_rate="auto", init='random', random_state=0).fit_transform(X_PCA)
# reducer = umap.UMAP(min_dist=0.9, n_components=60)
# X_UMAP = reducer.fit_transform(X_norm)
# X_batch = np.reshape(X_UMAP, (X_UMAP.shape[0]//5,5,X_UMAP.shape[1]))
# # X_UMAP_Norm = Normalizer().fit_transform(X_UMAP)

In [5]:
root_audios_Zamuro = f"{root}/Zamuro/Complementary_Files/zamuro_audios.csv"
root_recorders_Zamuro = f"{root}/Zamuro/Complementary_Files/zamuro_recorders.csv"

audios = pd.read_csv(root_audios_Zamuro, index_col=0)
recorders = pd.read_csv(root_recorders_Zamuro, index_col=0)

def combinar_nombre_ubicacion(row):
    return f"{row['field_number_PR']}_{row['Filename']}"

# Aplicando la función a cada fila del DataFrame para crear la nueva columna
audios['Filename_'] = audios.apply(combinar_nombre_ubicacion, axis=1)

audios.set_index("Filename_", inplace=True)

# AE Model

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

models = [10,20,30,40,50]

accuracies = []
f1_scores = []
recalls = []
for  i, mod_id in enumerate(models):

    X_ae = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/epochs_50/{mod_id}/AE_features_Zamuro.pth",  map_location=torch.device('cpu'))
    X_ae = np.asarray(X_ae)
    y_path = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/epochs_50/{mod_id}/AE_test_path_samples_Zamuro.pth",  map_location=torch.device('cpu'))
    
    y = np.asarray(y_path)
    y_2 = y[:,0]
    for i in range(len(y_2)):
        y_2[i] = y_2[i][0:-2] 
    y_2 = list(y_2)
    y_3 = np.repeat(y_2, 5)
    labels_ae = []
    for i in range(len(y_3)):
        labels_ae.append(audios.loc[y_3[i], "cover"])
        
    X_train, X_test, y_train, y_test = train_test_split(X_ae, labels_ae, test_size=0.2,random_state=0)
    
    clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
    clf_rf.fit(X_train, y_train)

    y_pred_rf = clf_rf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred_rf)
    f1_score = metrics.f1_score(y_test, y_pred_rf, average="macro")
    recall = metrics.recall_score(y_test, y_pred_rf, average="macro")
    print("Accuracy:", accuracy)
    print("f1:", f1_score)
    print("recall", recall)
    
    accuracies.append(accuracy)
    f1_scores.append(f1_score)
    recalls.append(recall)
    
np.save(f"{root}/Zamuro/temporal_zamuro/accuracies.npy", accuracies)
np.save(f"{root}/Zamuro/temporal_zamuro/f1_scores.npy", f1_scores)
np.save(f"{root}/Zamuro/temporal_zamuro/recalls.npy", recalls)

# X_ae = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/log_standarization_epochs_10/AE_features_Zamuro.pth",  map_location=torch.device('cpu'))
# X_ae = np.asarray(X_ae)
# y_path = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/log_standarization_epochs_10/AE_test_path_samples_Zamuro.pth",  map_location=torch.device('cpu'))

# scaler = StandardScaler().fit(X_ae)
# X_scaled = scaler.transform(X_ae)

# Normalizer_ = Normalizer().fit(X_ae)
# X_norm = Normalizer_.transform(X_ae)


In [6]:
y = np.asarray(y_path)
y_2 = y[:,0]
for i in range(len(y_2)):
    y_2[i] = y_2[i][0:-2] 
y_2 = list(y_2)
y_3 = np.repeat(y_2, 5)
labels_ae = []
for i in range(len(y_3)):
    labels_ae.append(audios.loc[y_3[i], "cover"])

### Autoencoders Features and Labels using independent segments

In [7]:
from sklearn import svm
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_ae, labels_ae, test_size=0.2,random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
clf_rf.fit(X_train, y_train)

y_pred_rf = clf_rf.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred_rf)
f1_score = metrics.f1_score(y_test, y_pred_rf, average="macro")
recall = metrics.recall_score(y_test, y_pred_rf, average="macro")
print("Accuracy:", accuracy)
print("f1:", f1_score)
print("recall", recall)

Accuracy: 0.7899145456781016
f1: 0.7188050972016402
recall 0.6935557365932045


### Autoencoders Features and Labels using 5 segments of the audio

In [8]:
X_batch = np.reshape(X_ae, (X_ae.shape[0]//5,5,X_ae.shape[1]))
y_batch = np.reshape(y_ae, (y_ae.shape[0]//5, 5))

NameError: name 'y_ae' is not defined

In [ ]:
y_3 == y_path[:,0]

In [ ]:
len(labels_ai), y_path[0].shape

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_batch, y_path, test_size=0.2,random_state=0)

X_train = X_train.reshape(X_train.shape[0]*X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0]*X_test.shape[1], X_test.shape[2])
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
y_train = y_train.reshape(y_train.shape[0]*y_train.shape[1])
y_test = y_test.reshape(y_test.shape[0]*y_test.shape[1])

labels_train = []
for i in range(len(y_train)):
    labels_train.append(audios.loc[y_train[i], "cover"])
labels_test = []
for i in range(len(y_test)):
    labels_test.append(audios.loc[y_test[i], "cover"])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
clf_rf.fit(X_train, labels_train)
y_pred_rf = clf_rf.predict(X_test)
accuracy = metrics.accuracy_score(labels_test, y_pred_rf)
f1_score = metrics.f1_score(labels_test, y_pred_rf, average="macro")
recall = metrics.recall_score(labels_test, y_pred_rf, average="macro")
print("Accuracy:", accuracy)
print("f1:", f1_score)
print("recall", recall)

# Autoencoders Features and Labels using voting labels

In [ ]:
y_pred_rf2 = np.asarray(y_pred_rf)
y_pred_rf2 = np.reshape(y_pred_rf2,(y_pred_rf2.shape[0]//5,5))
y_test2 = np.asarray(labels_test)
y_test2 = np.reshape(y_test2,(y_test2.shape[0]//5,5))

def most_frequent(List):
    return max(set(List), key = List.count)

labels_test2 = []
labels_pred = []
for i in range(len(y_pred_rf2)):
    labels_pred.append(most_frequent(list(y_pred_rf2[i])))
    labels_test2.append(most_frequent(list(y_test2[i])))
accuracy_ae = metrics.accuracy_score(labels_test2, labels_pred)
f1_score_ae = metrics.f1_score(labels_test2, labels_pred, average="macro")
recall_ae = metrics.recall_score(labels_test2, labels_pred, average="macro")

print("Accuracy:", accuracy_ae)
print("f1:", f1_score_ae)
print("recall", recall_ae)